<a href="https://colab.research.google.com/github/DeepOzean/generative_summarization/blob/gpt2/FineTune_GPT_2_nsheppered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scientific papers summarization(on Pubmed data)

Andrei Vlasov, contact: @ahlesen

Elizaveta Lazareva, contact: @elizaveta__

Nikolay Shvetsov, contact: @NikoShv

# Dear Instructor and TA's if you have any question according runinig any cell please dont hesitate contact us by email or telegram. 



# Introduction

Text summarization refers to the technique of shortening long pieces of text. The intention is to create a coherent and fluent summary having only the main points outlined in the document. Text summarization refers to the technique of shortening long pieces of text. The intention is to create a coherent and fluent summary having only the main points outlined in the document.

## Task
Produce abstractive summaries of
long scientific papers that exceed several thousand words via neural abstractive summarization. Our main task was create different models and compare their quality. The first model is GPT 2. We used pretrained model(OpenAi,https://openai.com/blog/gpt-2-1-5b-release/). Our second task was to train GPT 2 more on our data (Pubmed) and calculated main metrics ROUGE.  Also we decided take Discorse-Aware Attention model for Abstractive Summarization(https://arxiv.org/pdf/1804.05685.pdf). This model is not pretrained.The architecture of model is very difficult and we partly use original autorths code in order to implemend model in Pytorch framework and fit on our data. More technical detail we will describe in section Methods.As metric we use ROUGE

We partly used Tensorflow framework because pretrained model 

## Dataset
Our dataset is Pubmed, it is dataset created from medical papers. The data presented in json format. The tran dataset consist from 119 924 papers,test dataset from 6 658 papers and validation data from 6633 paper.Each paper consisit from several sections. The mean word's quantaty is 18 000 words. For abstract  it is 1 200 words. The abstract is our target. In this iteration we have 2 idea how to process data, the first is process sentences in paper if length of sentece is greater then 5 words, and limit for sentence length is 25 words. The second case consider only introduction(the mean length indroduction section is 3 000 words). We try 1st one, but the training process is very long, and in 5 000 iteration we have not complete, were meaningless. For our opinion, as we use huge corpus we have not well trained model for such amount of data. That's why we decided to use only introduction section, and in 5 000 iteration we have satisfying results. We generated 5 reviews, and about 3 review is good in average

In [0]:
# %cd /content/gdrive/My Drive/NLP/SkolTech/Project/gpt-2
# %ls


/content/gdrive/My Drive/NLP/SkolTech/Project/gpt-2
'=0.1.3'
'Copy of Project.ipynb'
 FineTune_GPT_2_Introduction_final.ipynb
 FineTune_GPT_2_Introduction.ipynb
 FineTune_GPT_2_MY_dataset-Copy1.ipynb
 FineTune_GPT_2_Vlasov.ipynb
 gpt-2/
 Project_Summarization.ipynb
 pubmed-release/
 review_and_summary_arxiv.txt
 review_and_summary_pubmed.txt
 review_test_pubmed.txt
 summary_test_pubmed.txt
'Копия FineTune_GPT_2_Introduction.ipynb'
'Копия Project_Summarization.ipynb'


In [1]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# %matplotlib inline
# %cd /content/gdrive/My Drive/Colab Notebooks/gpt-2 #Nikolay

# # %cd /content/gdrive/My Drive/NLP #ANdrey
# # %cd SkolTech/Project/gpt-2 #ANdrey

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
[Errno 2] No such file or directory: '/content/gdrive/My Drive/Colab Notebooks/gpt-2 #Nikolay'
/content


In [0]:
import requests
from tqdm import tqdm

# Functions to load dataset from google drive

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None
        
def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in tqdm(response.iter_content(CHUNK_SIZE)):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
def download_file_from_google_drive(id, destination):

    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        save_response_content(response, destination)


In [0]:
# Load PubMed data
# !!!!!! Takes lots of time !!!!!!

download_file_from_google_drive("1Sa3kip8IE0J1SkMivlgOwq1jBgOnzeny", "data.zip")

In [0]:
# Unzip data
# !!!!!! Takes lots of time !!!!!!

!unzip data.zip

In [0]:
# Load GPT-2 models, weights and predicted summurizes, it's takes a lots of time!!!!
download_file_from_google_drive("1wc0DPGrtPTrcxPH4sBPnQNpD6Sw1Rwl-", "gpt-2.zip")

In [0]:
#loads models, a lot of time!!!!
!unzip gpt-2.zip

In [0]:
# As we dont know which libraries you have, we install all dependences, if you have those, please delete
!pip install fire>=0.1.3
!pip install regex==2017.4.5
!pip install requests==2.21.0
!pip install tqdm==4.31.1
!pip install numpy==1.17.4
!pip install pandas==0.25.3
!pip install tensorflow
!pip install toposort
!pip install rouge
!pip install tqdm==4.36.1
import json
import gzip
import math
import numpy as np
import pandas as pd
import string
import re
from tqdm import tqdm
import os
import sys
from rouge import Rouge 
import time
from time import gmtime, strftime
import torch
from torch.nn.utils import clip_grad_norm_
from torch.optim import Adam, Adagrad 
from torch.optim.lr_scheduler import StepLR
# !pip install -r requirements.txt

## Data preprocessing

The data presented in json format, some paper didnt consist text in section: article text, some paper didnt have abstract or have no quality abstract, or abstract very small size(less then sentence). In this way we didnt consider those papers.   

In [0]:
def read_data(path,dir):
    """
    read data in list
    :path - path to file
    :dir - path to dir
    """
    data_path = f'{dir}/'
    data_path += path
    data= []
    with open(data_path) as file:
        for line in file:
            data.append(line.strip())
    return data
train_jsons, test_jsons, val_jsons = read_data('train.txt','pubmed-release'),read_data('test.txt','pubmed-release'),read_data('val.txt','pubmed-release')
train_jsons = [json.loads(train) for train in train_jsons]
val_jsons = [json.loads(val) for val in val_jsons]
test_jsons = [json.loads(test) for test in test_jsons]


In [0]:
def preprocess(train_jsons):
    '''
    remove  articles without introduction and abstract part
    :train_jsons - json file for preprocessing
    '''
    article_text = []
    abstract_text = []
    article_id = []
    for json_1 in train_jsons:     
        if len(json_1['sections'][0]) < 2:
          continue 
        article_text.append("".join(json_1['sections'][0]).replace('\n',''))#'article_text'
        abstract_text.append("".join(json_1['abstract_text']).replace('<S>','').replace(' </S>',''))
    return article_text,abstract_text

In [0]:
# preprocess train and test data
article_text_train,abstract_text_train = preprocess(train_jsons)
article_text_test,abstract_text_test = preprocess(test_jsons)
# length of train and test data 
len(article_text_train),len(article_text_test)

(112744, 6422)

We tried to retrain our model on more data but collab doesn't have enough memory. 

Also, we take only 1000 targets for making reviews because it wastes a lot of time.

In [0]:
article_text_train = article_text_train[:40000]
abstract_text_train = abstract_text_train[:40000]

In [0]:
article_text_test = article_text_test[:1000]
abstract_text_test = abstract_text_test[:1000]

In [0]:
%cd gpt-2/

/content/gdrive/My Drive/NLP/SkolTech/Project/gpt-2/gpt-2


Make dataset with reviews (introduction) and summary (abstractive) for training

In [0]:
corpus = ''
for i in range(len(article_text_train)):
    corpus += article_text_train[i]
    corpus += ' TL;DR: '
    corpus += abstract_text_train[i]
    corpus += ' [End] '
    corpus += '\n'
    if i <5:
        print(corpus)

with open('review_and_summary_pubmed_intro.txt', 'w') as f:
    f.write(corpus)

a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries .in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively .the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% .anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight .snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united statesthere are also some reports regarding school feeding programs in developing countries . in vietnam ,school

In [0]:
import src.model as model
import src.sample as sample
import src.encoder as encoder
from src.load_dataset import load_dataset, Sampler
from src.accumulate import AccumulatingOptimizer
import src.memory_saving_gradients as memory_saving_gradients
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2
import time


import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
def encode_to_npz(in_text,out_npz,model_name='117M',combine=50000):
    """
    rewrite file from txt format to npz with separator adding
    :model_name - Pretrained model name
    :combine - Concatenate files with <|endoftext|> separator into chunks of this minimum size
    :in_text - Input file, directory, or glob pattern (utf-8 text)
    :out_npz - Output file path
    """
    enc = encoder.get_encoder(model_name)
    print('Reading files')
    chunks = load_dataset(enc, in_text, combine)
    print('Writing', out_npz)
    np.savez_compressed(out_npz, *chunks)

In [0]:
in_text = 'review_and_summary_pubmed_intro.txt'
out_npz = in_text+'.npz'
encode_to_npz(in_text,out_npz,model_name='117M',combine=50000)



  0%|          | 0/1 [00:00<?, ?it/s]

Reading files
start bpe


final bpe



100%|██████████| 1/1 [03:24<00:00, 204.90s/it]


Writing review_and_summary_pubmed_intro.txt.npz


Make dataset with reviews (introduction) for testing



In [0]:
corpus_test = ''
for review in article_text_test:
  corpus_test += review
  corpus_test += ' TL;DR:\n'

with open('review_test_pubmed_intro.txt', 'w') as f:
  f.write(corpus_test)

Test that we don't have empty spaces in test reviews

In [0]:
with open('review_test_pubmed_intro.txt','r') as file:
  for num,line in enumerate(file):
    if len(line) < 6:
      print('WARNING!'*3)
      print(num,len(line),line)

Make dataset with summaries (abstractive) for testing


In [0]:
summary_test = ''
for summary in abstract_text_test:
  summary_test += summary
  summary_test += '\n'

with open('summary_test_pubmed_intro.txt', 'w') as f:
  f.write(summary_test)

# **Train**

> Firstly, we will retrain gpt-2 model 

> we will train a discourse model in A Discourse-Aware Attntion model Description

### **Finetune gpt-2 model**

In [0]:
CHECKPOINT_DIR = 'models'
SAMPLE_DIR = 'samples'

In [0]:
def maketree(path):
    try:
        os.makedirs(path)
    except:
        pass

def train(dataset, model_name, combine=50000,
          batch_size=1, learning_rate=0.0001, accumulate_gradients=1, memory_saving_gradients_flag=False, only_train_transformer_layers=False,
          restore_from='latest',run_name='run1',sample_every=100,sample_length=1023,sample_num=1,save_every=1000,
          val_dataset=None,val_batch_size=2,val_batch_count=40,val_every=0,stop_after=None):
    # args = parser.parse_args()
    enc = encoder.get_encoder(model_name)
    hparams = model.default_hparams()
    with open(os.path.join('models', model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if sample_length > hparams.n_ctx:
        raise ValueError(
            "Can't get samples longer than window size: %s" % hparams.n_ctx)

    if  '345M' in model_name:
        memory_saving_gradients_flag = True
        only_train_transformer_layers = True

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    config.graph_options.rewrite_options.layout_optimizer = rewriter_config_pb2.RewriterConfig.OFF

    #tf.reset_default_graph() 
    with tf.Session(config=config) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        output = model.model(hparams=hparams, X=context)
        loss = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=context[:, 1:], logits=output['logits'][:, :-1]))

        if val_every > 0:
            val_context = tf.placeholder(tf.int32, [val_batch_size, None])
            val_output = model.model(hparams=hparams, X=val_context)
            val_loss = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=val_context[:, 1:], logits=val_output['logits'][:, :-1]))
            val_loss_summary = tf.summary.scalar('val_loss', val_loss)


        tf_sample = sample.sample_sequence(
            hparams=hparams,
            length=sample_length,
            context=context,
            batch_size=batch_size,
            temperature=1.0,
            top_k=40)

        all_vars = [v for v in tf.trainable_variables() if 'model' in v.name]
        train_vars = [v for v in all_vars if '/h' in v.name] if only_train_transformer_layers else all_vars
        if accumulate_gradients > 1:
            if memory_saving_gradients_flag:
                exit("Memory saving gradients are not implemented for gradient accumulation yet.")
            opt = AccumulatingOptimizer(
                opt=tf.train.AdamOptimizer(learning_rate=learning_rate),
                var_list=train_vars)
            opt_reset = opt.reset()
            opt_compute = opt.compute_gradients(loss)
            opt_apply = opt.apply_gradients()
            summary_loss = tf.summary.scalar('loss', opt_apply)
        else:
            opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
            if memory_saving_gradients_flag:
                opt_grads = memory_saving_gradients.gradients(loss, train_vars)
            else:
                opt_grads = tf.gradients(loss, train_vars)
            opt_grads = list(zip(opt_grads, train_vars))
            opt_apply = opt.apply_gradients(opt_grads)
            summary_loss = tf.summary.scalar('loss', loss)

        summary_log = tf.summary.FileWriter(
            os.path.join(CHECKPOINT_DIR, run_name))

        saver = tf.train.Saver(
            var_list=all_vars,
            max_to_keep=5,
            keep_checkpoint_every_n_hours=2)
        sess.run(tf.global_variables_initializer())

        if restore_from == 'latest':
            ckpt = tf.train.latest_checkpoint(
                os.path.join(CHECKPOINT_DIR, run_name))
            if ckpt is None:
                # Get fresh GPT weights if new run.
                ckpt = tf.train.latest_checkpoint(
                    os.path.join('models', model_name))
        elif restore_from == 'fresh':
            ckpt = tf.train.latest_checkpoint(
                os.path.join('models', model_name))
        else:
            ckpt = tf.train.latest_checkpoint(restore_from)
        print('Loading checkpoint', ckpt)
        saver.restore(sess, ckpt)

        print('Loading dataset...')
        chunks = load_dataset(enc, dataset, combine)
        data_sampler = Sampler(chunks)
        if val_every > 0:
            val_chunks = load_dataset(enc, val_dataset, combine) if val_dataset else chunks
        print('dataset has', data_sampler.total_size, 'tokens')
        print('Training...')

        if val_every > 0:
            # Sample from validation set once with fixed seed to make
            # it deterministic during training as well as across runs.
            val_data_sampler = Sampler(val_chunks, seed=1)
            val_batches = [[val_data_sampler.sample(1024) for _ in range(val_batch_size)]
                           for _ in range(val_batch_count)]

        counter = 1
        counter_path = os.path.join(CHECKPOINT_DIR, run_name, 'counter')
        if os.path.exists(counter_path):
            # Load the step number if we're resuming a run
            # Add 1 so we don't immediately try to save again
            with open(counter_path, 'r') as fp:
                counter = int(fp.read()) + 1

        def save():
            maketree(os.path.join(CHECKPOINT_DIR, run_name))
            print(
                'Saving',
                os.path.join(CHECKPOINT_DIR, run_name,
                             'model-{}').format(counter))
            saver.save(
                sess,
                os.path.join(CHECKPOINT_DIR, run_name, 'model'),
                global_step=counter)
            with open(counter_path, 'w') as fp:
                fp.write(str(counter) + '\n')

        def generate_samples():
            print('Generating samples...')
            context_tokens = data_sampler.sample(1)
            all_text = []
            index = 0
            while index < sample_num:
                out = sess.run(
                    tf_sample,
                    feed_dict={context: batch_size * [context_tokens]})
                for i in range(min(sample_num - index, batch_size)):
                    text = enc.decode(out[i])
                    text = '======== SAMPLE {} ========\n{}\n'.format(
                        index + 1, text)
                    all_text.append(text)
                    index += 1
            print(text)
            maketree(os.path.join(SAMPLE_DIR, run_name))
            with open(
                    os.path.join(SAMPLE_DIR, run_name,
                                 'samples-{}').format(counter), 'w') as fp:
                fp.write('\n'.join(all_text))

        def validation():
            print('Calculating validation loss...')
            losses = []
            for batch in tqdm.tqdm(val_batches):
                losses.append(sess.run(val_loss, feed_dict={val_context: batch}))
            v_val_loss = np.mean(losses)
            v_summary = sess.run(val_loss_summary, feed_dict={val_loss: v_val_loss})
            summary_log.add_summary(v_summary, counter)
            summary_log.flush()
            print(
                '[{counter} | {time:2.2f}] validation loss = {loss:2.2f}'
                .format(
                    counter=counter,
                    time=time.time() - start_time,
                    loss=v_val_loss))

        def sample_batch():
            return [data_sampler.sample(1024) for _ in range(batch_size)]


        avg_loss = (0.0, 0.0)
        start_time = time.time()

        try:
            while counter != stop_after:
                if counter % save_every == 0:
                    save()
                if counter % sample_every == 0:
                    generate_samples()
                if val_every > 0 and (counter % val_every == 0 or counter == 1):
                    validation()

                if accumulate_gradients > 1:
                    sess.run(opt_reset)
                    for _ in range(accumulate_gradients):
                        sess.run(
                            opt_compute, feed_dict={context: sample_batch()})
                    (v_loss, v_summary) = sess.run((opt_apply, summary_loss))
                else:
                    (_, v_loss, v_summary) = sess.run(
                        (opt_apply, loss, summary_loss),
                        feed_dict={context: sample_batch()})

                summary_log.add_summary(v_summary, counter)

                avg_loss = (avg_loss[0] * 0.99 + v_loss,
                            avg_loss[1] * 0.99 + 1.0)

                print(
                    '[{counter} | {time:2.2f}] loss={loss:2.2f} avg={avg:2.2f}'
                    .format(
                        counter=counter,
                        time=time.time() - start_time,
                        loss=v_loss,
                        avg=avg_loss[0] / avg_loss[1]))

                counter += 1
        except KeyboardInterrupt:
            print('interrupted')
        finally:
            save()

In [0]:
your_model_train = '117_M_pubmed_intro'
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! if you start train and loss weigths, you can find original model in 117_M_pubmed_intro_original!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [0]:
train(dataset='review_and_summary_pubmed_intro.txt.npz', model_name='117M', 
          batch_size=2, learning_rate=1e-4, run_name=your_model_train,sample_every=100,stop_after=100)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
Loading checkpoint models/117M/model.ckpt
INFO:tensorflow:Restoring parameters from models/117M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:01<00:00,  1.34s/it]


dataset has 36259517 tokens
Training...
[1 | 4.98] loss=3.30 avg=3.30
[2 | 5.46] loss=3.46 avg=3.38
[3 | 5.93] loss=3.25 avg=3.34
[4 | 6.41] loss=3.38 avg=3.35
[5 | 6.89] loss=2.96 avg=3.27
[6 | 7.37] loss=3.18 avg=3.25
[7 | 7.85] loss=3.17 avg=3.24
[8 | 8.33] loss=3.41 avg=3.26
[9 | 8.81] loss=3.68 avg=3.31
[10 | 9.29] loss=3.27 avg=3.31
[11 | 9.76] loss=2.92 avg=3.27
[12 | 10.24] loss=3.32 avg=3.27
[13 | 10.72] loss=3.23 avg=3.27
[14 | 11.20] loss=3.28 avg=3.27
[15 | 11.68] loss=3.29 avg=3.27
[16 | 12.16] loss=3.15 avg=3.26
[17 | 12.64] loss=3.24 avg=3.26
[18 | 13.12] loss=2.70 avg=3.23
[19 | 13.60] loss=3.43 avg=3.24
[20 | 14.08] loss=3.48 avg=3.25
[21 | 14.55] loss=3.46 avg=3.26
[22 | 15.03] loss=3.14 avg=3.26
[23 | 15.51] loss=3.53 avg=3.27
[24 | 15.99] loss=3.18 avg=3.27
[25 | 16.47] loss=3.26 avg=3.27
[26 | 16.95] loss=3.35 avg=3.27
[27 | 17.43] loss=3.07 avg=3.26
[28 | 17.90] loss=3.42 avg=3.27
[29 | 18.38] loss=3.49 avg=3.28
[30 | 18.86] loss=3.37 avg=3.28
[31 | 19.34] loss=3.

In [0]:
!ls models/117_M_pubmed_intro

checkpoint				     model-3000.data-00000-of-00001
counter					     model-3000.index
encoder.json				     model-3000.meta
events.out.tfevents.1576699588.0cec2feb27b4  model-4000.data-00000-of-00001
hparams.json				     model-4000.index
model-1000.data-00000-of-00001		     model-4000.meta
model-1000.index			     model-5000.data-00000-of-00001
model-1000.meta				     model-5000.index
model-2000.data-00000-of-00001		     model-5000.meta
model-2000.index			     vocab.bpe
model-2000.meta


# Test

### 1. make test summaries on finetune gpt-2

### 2. make test summaries on gpt-2

### In the end of A Discourse-Aware Attention model description we make test summaries on a discourse.  After that we evaluate and compare our results.

### Test summaries on finetune gpt-2

In [0]:
def interact_model(
    reviewText,
    summaries,
    json_name = 'generated_summaries',
    model_name='117M',
    seed=None,
    nsamples=1,
    batch_size=1,
    length=None,
    temperature=1,
    top_k=0,
    silent=0
):
    """ take comment from https://openai.com/blog/gpt-2-1-5b-release/
    Interactively run the model
    :model_name=117M : String, which model to use
    :seed=None : Integer seed for random number generators, fix seed to reproduce
     results
    :nsamples=1 : Number of samples to return total
    :batch_size=1 : Number of batches (only affects speed/memory).  Must divide nsamples.
    :length=None : Number of tokens in generated text, if None (default), is
     determined by model hyperparameters
    :temperature=1 : Float value controlling randomness in boltzmann
     distribution. Lower temperature results in less random completions. As the
     temperature approaches zero, the model will become deterministic and
     repetitive. Higher temperature results in more random completions.
    :top_k=0 : Integer value controlling diversity. 1 means only 1 word is
     considered for each step (token), resulting in deterministic completions,
     while 40 means 40 words are considered at each step. 0 (default) is a
     special setting meaning no restrictions. 40 generally is a good value.
    """
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name)
    hparams = model.default_hparams()
    with open(os.path.join('models', model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join('models', model_name))
        saver.restore(sess, ckpt)

        json_dict = {}
        with open(reviewText, 'r') as f:
            with open(summaries, 'r') as g:
                reviewText_corpus = f.readlines()
                summary_corpus = g.readlines()
                review_no = 0
                for review, summary in zip(reviewText_corpus,summary_corpus):
                    print("=" * 40 + " REVIEW " + str(review_no) + " " + "=" * 40)
                    if (not silent) and (review_no <50):
                        print(review)
                        print("=" * 40 + " GROUND TRUTH " + str(review_no) + " " + "=" * 40)
                        print(summary)
                    review = review[:-1]
                    review_no += 1
                    json_review = {}
                    json_review['reviewText'] = review
                    json_review['summary'] = summary
                    context_tokens = enc.encode(review)
                    generated = 0
                    generated_summary_list = []
                    for _ in range(nsamples // batch_size):
                        out = sess.run(output, feed_dict={
                            context: [context_tokens for _ in range(batch_size)]
                        })[:, len(context_tokens):]
                        for i in range(batch_size):
                            generated += 1
                            text = enc.decode(out[i])
                            sep = ' [End]'
                            text = text.split(sep,1)[0]
                            if (not silent) and (review_no <50):
                                print("=" * 40 + " SUMMARY " + str(generated) + " " + "=" * 40)
                                print(text)
                            generated_summary_list.append(text)
                    if (not silent) and (review_no <50):
                        print("=" * 80)
                        print('\n\n')
                    json_review['generated'] = generated_summary_list
                    json_dict['review'+str(review_no)] = json_review
        with open(json_name+".json", "w") as write_file:
            json.dump(json_dict, write_file, indent=4)

In [0]:
!ls -lt models/117_M_pubmed_intro

total 490658
-rw------- 1 root root   1042301 Dec 19 22:18  encoder.json
-rw------- 1 root root        90 Dec 19 22:18  hparams.json
-rw------- 1 root root    456318 Dec 19 22:17 'Копия vocab.bpe'
-rw------- 1 root root         4 Dec 19 22:00  counter
-rw------- 1 root root   3167183 Dec 19 22:00  model-100.meta
-rw------- 1 root root        75 Dec 19 22:00  checkpoint
-rw------- 1 root root 497759232 Dec 19 22:00  model-100.data-00000-of-00001
-rw------- 1 root root      5215 Dec 19 22:00  model-100.index
-rw------- 1 root root        82 Dec 19 21:59  events.out.tfevents.1576792771.d5e4bb8f3889


In [0]:
interact_model(reviewText='review_test_pubmed_intro.txt',model_name=your_model_train,summaries='summary_test_pubmed_intro.txt',json_name='pubmed_reviews_on_train_info',
               top_k=30,temperature=0.7,length=100,nsamples=5,silent=0,batch_size=1)

INFO:tensorflow:Restoring parameters from models/117M_pubmed_intro_train_kek/model-100
======================================== REVIEW 0 ========================================
anxiety affects quality of life in those living with parkinson 's disease ( pd ) more so than overall cognitive status , motor deficits , apathy , and depression [ 13 ] .although anxiety and depression are often related and coexist in pd patients , recent research suggests that anxiety rather than depression is the most prominent and prevalent mood disorder in pd [ 5 , 6 ] . yet ,our current understanding of anxiety and its impact on cognition in pd , as well as its neural basis and best treatment practices , remains meager and lags far behind that of depression .overall , neuropsychiatric symptoms in pd have been shown to be negatively associated with cognitive performance .for example , higher depression scores have been correlated with lower scores on the mini - mental state exam ( mmse ) [ 8 , 9 ] as well a


final bpe
======================================== SUMMARY 1 ========================================
  anxiety and depression are often coexist in pd , so we hypothesized that pd patients with anxiety and depression would show impairments in attentional set - shifting and working memory compared to pd without anxiety . to date , there have been few studies examining the relationship between anxiety and cognition in pd , as well as the impact of anxiety on cognition in pd , and it has been hypothesized that anxiety significantly impacts cognitive performance in pd patients with anxiety and depression in comparison to pd
======================================== SUMMARY 2 ========================================
  attentional processing in pd patients is adversely affected , and cognitive processing is impaired , and anxiety is a common symptom , whereas depression is not . while depressive symptoms are more frequently present in pd patients with anxiety , the effects of anxiety on cogn


final bpe
======================================== SUMMARY 1 ========================================
  cancer is caused by multiple processes , including uncontrolled cellular proliferation and inappropriate survival . in the past two decades , several recent studies have demonstrated the role of mirna - encoded rna , which is characterized by the presence or absence of an in vivo tumor suppressor . in the present study , we report on a recent discovery in the mammary gland tumorigenesis of rna - encoded rna , which could represent a novel cancer therapy . 
 a recent discovery in the mammary gland tumorigen
======================================== SUMMARY 2 ========================================
  background -  background -  background -  background -  background - background - background - background - background - background - background - background - background - background - background - background - background - background - background - background - background - background -


final bpe
======================================== SUMMARY 1 ========================================
 a pigtail catheck -   ( pigtail catheter  ( pigtail ( pigtail ( pigtail or pigtail ) catheter tail  pigtail tail tail tail tail tail tail  tail tail tail tail tail tail tail tail tail tail  tail tail tail  tail tail tail  tail tail tail tail tail tail tail tail tail tail
======================================== SUMMARY 2 ========================================
 the pigtail catheter drainage , the pigtail catheter drainage and percut - drainage in the catheter drainage , percutaneous catheter drainage and catheter drainage and catheter drainage , catheter drainage and catheter drainage [catheter drainage and catheter drainage and catheter drainage . inpatient catheter - catheter drainage inpatient catheter drainage , inpatient - catheter drainage and intheating [catheter drainage and catheter drainage and female inthealing inpatient 
======================================== SUMMARY 3


final bpe
======================================== SUMMARY 1 ========================================
  adrenal hyperplasia ( cah ) is a group of autosomal recessive disorders caused by an enzyme deficiency which leads to defects in biosynthesis of steroid precursors . in the past few years , we have seen a rapid rise in the prevalence of cah and this prevalence has increased dramatically in recent years . in the past few months , we have seen a rapid rise in the prevalence of cah and this prevalence has increased dramatically in recent years . although the prevalence of cah has
======================================== SUMMARY 2 ========================================
  it is extremely rare for a congenital adrenal hyperplasia ( cah ) to cause a premature birth. however , in many cases , the prognosis for the mothers is very good . in the case of the mothers born in the late 1980s , the prognosis is very good . in this context , the prognosis for the babies is very good . in the case

...


final bpe
======================================== REVIEW 407 ========================================
start bpe



final bpe
======================================== REVIEW 408 ========================================
start bpe



final bpe
======================================== REVIEW 409 ========================================
start bpe



final bpe


KeyboardInterrupt: ignored

In [0]:
!ls models/117M

checkpoint    hparams.json		      model.ckpt.index	vocab.bpe
encoder.json  model.ckpt.data-00000-of-00001  model.ckpt.meta


## Test summaries on gpt-2 (without fine-tuning)


In [0]:
interact_model(reviewText='review_test_pubmed_intro.txt',model_name='117M',summaries='summary_test_pubmed_intro.txt',json_name='pubmed_reviews_117M',top_k=30,temperature=0.7,length=100,nsamples=5,silent=1,batch_size=1)







Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.

INFO:tensorflow:Restoring parameters from models/117M/model.ckpt
======================================== REVIEW 0 ========================================
start bpe



final bpe
======================================== REVIEW 1 ========================================
start bpe



final bpe
======================================== REVIEW 2 ========================================
start bpe



final bpe
======================================== REVIEW 3 ========================================
start bpe



final bpe
======================================== REVIEW 4 ========================================
start bpe


...


final bpe
======================================== REVIEW 17 ========================================
start bpe



final bpe
======================================== REVIEW 18 ========================================
start bpe



final bpe
======================================== REVIEW 19 ========================================
start bpe



final bpe
======================================== REVIEW 20 ========================================
start bpe



final bpe


#A Discourse-Aware Attention Model for Abstractive Summarization of Long Documents

here we have not outputs, because we build notebook from 2 separate and dont have time run everything in one. But we load all weights , outputs and check metrics, if you want to train by yourself, you are welcome

A Discourse-Aware Attention Model includes a hierarchical encoder, capturing the discourse structure of the document and a discourse-aware decoder that generates the summary. This decoder attends to different discourse sections and allows the model to more accurately represent important information from the source resulting in a better context vector.

In [0]:
!pip install torch==1.3.0

In [0]:
%cd ../

In [0]:
# Cloning from OUR repository files for training 
# !!!!!! Takes 3-5 minutes !!!!!!

! git clone https://github.com/liza9797/Abstractive-Summarization-pytorch.git

In [0]:
# Install git
! cd Abstractive-Summarization-pytorch; git lfs install
# Loading weights
! cd Abstractive-Summarization-pytorch; git lfs fetch; git lfs checkout

In [0]:
sys.path.insert(0, "Abstractive-Summarization-pytorch")

import config
from data import Vocab, abstract2sents
from batcher import Example, Batch
from model import Model

from utils import get_input_from_batch, get_output_from_batch

In [0]:
def preprocessing_file(file_path):
    r'''The function for parsing the PubMed files. It returns a list of number of samples 
    in the data lenth. The items of the list are dictionaries with different features. One 
    item corresponds to one sample in the data'''
    
    # Create name of features of each sample
    keys = ['article_id', 'article_text', 'abstract_text', 'labels', 'section_names', 'sections']
    content_list = []

    with open(file_path, "r") as f:
        # Iterate over the file
        for l in tqdm(f.readlines()):
            
            # Split data by the '\' symbol
            content = [item for item in l.split("\"")]
            item_dict = {}

            # Parsing data
            for item in content[1:]:
                
                # If the word is the name of feature, create the list for this feature
                if item in keys:
                    item_dict[item] = []
                    key_ = item
                    if item == 'sections':
                        count_sections = -1
                        
                # If it is the word, add to the list
                else:
                    # Do not add the separation symbols, which separate sentances 
                    # and blocks of different features
                    if key_ != 'sections':
                        if item not in ['], ', ', ', ': [[', ']]}\n', '{', ': [', ': '] :
                            item_dict[key_].append(item)
                    else:
                        if item in [': [[',  '], [']:
                            item_dict[key_].append([])
                            count_sections += 1
                        elif item not in [']]}\n', ', ' ]:
                            item_dict[key_][count_sections].append(item)
            
            # If article is empty, we remove this sample from the data
            if len(item_dict['article_text']) > 1:
                item_dict['abstract_text'] = " ".join(item_dict['abstract_text'] )
                item_dict['abstract_text'] = [sent.strip() for sent in abstract2sents(item_dict['abstract_text'])]
                content_list.append(item_dict)

        
    return content_list

In [0]:
# Load data

data_ = {}
data_["train"] = preprocessing_file("pubmed-release/train.txt")
data_["val"] = preprocessing_file("pubmed-release/val.txt")
# Create the special class for containing Vocabular
vocab = Vocab("pubmed-release/vocab", 50100)

In [0]:
# Exaple of word to index method applyed to word 'medical'
vocab.word2id("medical")

# Batcher class

In [0]:
class Batcher(object):
    r''' The class which applies all transforms to input data using Example and Batch classes 
    (which are applied sequentionally). 
    
    Class Example do the following with each imput sample:
     - take 400 first words of the article text, transform it into the list of indices;
     - take 100 first words of the abstract text, transform it into the list of indices.
     
    Class Batch do the following with each imput sample:
     - form the tensors of batch size of given input list of Example samples.
     
    '''
    def __init__(self, data_dicts, vocab, batch_size, shuffle=True):
        self.data_dicts = data_dicts
        self.vocab = vocab
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        if self.shuffle:
            inds = np.arange(len(self.data_dicts))
            np.random.shuffle(inds)
            self.data_dicts = [self.data_dicts[ind] for ind in list(inds)]
            
        self.start_batch = 0
    def next_batch(self):
        
        if self.start_batch + self.batch_size >= len(self.data_dicts):
            return None

        example_list = [Example(article=" ".join(self.data_dicts[self.start_batch + i]['article_text']), 
                          abstract_sentences=self.data_dicts[self.start_batch + i]['abstract_text'], 
                          vocab=self.vocab) 
                        for i in range(self.batch_size)]
        
        batch = Batch(example_list=example_list, 
                      vocab=self.vocab, 
                      batch_size=self.batch_size) 
        
        self.start_batch += self.batch_size
        return batch

In [0]:
def calc_running_avg_loss(loss, running_avg_loss, step, decay=0.99):
    if running_avg_loss == 0:  # on the first iteration just take the loss
        running_avg_loss = loss
    else:
        running_avg_loss = running_avg_loss * decay + (1 - decay) * loss
    running_avg_loss = min(running_avg_loss, 12)  # clip

    return running_avg_loss

def train_model(model, batcher, optimizer, scheduler, N_epoch, use_cuda=False, start_iter=0, 
                running_avg_loss=0, toy_example=False):
    r''' This function create a folder "train_CURRENT_TIME/" in the "weights/" folder, 
    where it will save all weights every 1000 iterations with the training history file, 
    trains the model with the given optimizer and scheduler for N_epoch epoches.
    '''
    train_history = {}
    df_history = pd.DataFrame(columns=["epoch", "itr", "loss", "running_loss"])
    
    train_dir = "weights/train_{}".format(strftime("%Y-%m-%d_%H:%M:%S", gmtime()))
    os.makedirs(train_dir)

    for epoch in range(N_epoch):
        train_history[epoch] = {"loss":[], "running_loss":[]}
        itr = 0
        
        # Save model
        while True:
            if itr % 1000 == 999:
                try:
                    torch.save({
                        'epoch': epoch,
                        'itr': itr,
                        'encoder': model.encoder.state_dict(),
                        'decoder': model.decoder.state_dict(),
                        'reduce_state': model.reduce_state.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss}, train_dir + "/weights.loss_{:.3f}.pt".format(loss))
                except:
                    print("Failed to save")

                scheduler.step()

            batch = batcher.next_batch()
            if batch:
                # Train model with given batch
                # Calculate loss
                loss = train_one_batch(model, optimizer, batch, use_cuda)
                train_history[epoch]["loss"].append(loss)
                
                # Calculate running average loss
                running_avg_loss = calc_running_avg_loss(loss, running_avg_loss, iter)
                train_history[epoch]["running_loss"].append(running_avg_loss)

                # Write in training history file
                df_history.loc[df_history.shape[0]] = [epoch+1, itr+1, loss, running_avg_loss]
                df_history.to_csv(train_dir + "/train_history.csv", index=False)

                print("{} epoch, {} itr: loss = {}".format(epoch +1, itr+1, running_avg_loss))

                itr += 1
            else:
                batcher.start_batch = 0
                break
                
            if toy_example:
                break
        if toy_example:
            break
    return model, train_history

In [0]:
def train_one_batch(model, optimizer, batch, use_cuda=False):
    r''' This function train the model on one batch.
    '''
    
    # From given batch we extract all items of article
    ### 'enc_batch' - tensor of size batch_size x 400, where the texts are presented as
    ###               a bag of indecies
    ### 'enc_lens' - the list text lens (if they are bigger then 400, then the len is 400)
    
    enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_1, coverage = \
        get_input_from_batch(batch, use_cuda)
    
    # From given batch we extract all items of abstract
    ### 'dec_batch' - tensor of size batch_size x 100, where the abstracts are presented as
    ###               a bag of indecies
    ### 'target_batch' - the same tensor as 'dec_batch', 
    
    dec_batch, dec_padding_mask, max_dec_len, dec_lens_var, target_batch = \
        get_output_from_batch(batch, use_cuda)
    
    optimizer.zero_grad()

    # Step 1. Encoding the batch, obtain:
    ### 'encoder_outputs': size = (batch_size x lenth x 128)
    ### 'encoder_feature': size = (batch_size x lenth x 512)
    ### 'encoder_hidden': tuple of vectors of size = (1 x batch_size x 512)
    
    encoder_outputs, encoder_feature, encoder_hidden = model.encoder(enc_batch, enc_lens) 
    
    # Step 2. Reduce the dimensionality of hidden state.
    ### 's_t_1': tuple of vectors of size = (1 x batch_size x 256)
    s_t_1 = model.reduce_state(encoder_hidden)

    step_losses = []
    # Step 3. Iterate over the number of words in the target and make predictions
    for di in range(min(max_dec_len, config.max_dec_steps)):
        
        # t-1-th word from the target which used as an input to predict t-th word
        y_t_1 = dec_batch[:, di] 
        
        # Prediction
        ### 'final_dist': size = (batch_size x vocab_size) - the probabilities of words
        
        final_dist, s_t_1,  c_t_1, attn_dist, p_gen, next_coverage = model.decoder(y_t_1, s_t_1,
                                                    encoder_outputs, encoder_feature, enc_padding_mask, c_t_1,
                                                    extra_zeros, enc_batch_extend_vocab,
                                                                       coverage, di)
        # t-th target word
        target = target_batch[:, di]
        
        # NLL loss
        gold_probs = torch.gather(final_dist, 1, target.unsqueeze(1)).squeeze()
        step_loss = -torch.log(gold_probs + config.eps)
        if config.is_coverage:
            step_coverage_loss = torch.sum(torch.min(attn_dist, coverage), 1)
            step_loss = step_loss + config.cov_loss_wt * step_coverage_loss
            coverage = next_coverage

        step_mask = dec_padding_mask[:, di]
        step_loss = step_loss * step_mask
        step_losses.append(step_loss)

    # Sum of losses for the batch
    sum_losses = torch.sum(torch.stack(step_losses, 1), 1)
    batch_avg_loss = sum_losses/dec_lens_var
    loss = torch.mean(batch_avg_loss)

    # Backward step
    loss.backward()

    norm = clip_grad_norm_(model.encoder.parameters(), config.max_grad_norm)
    clip_grad_norm_(model.decoder.parameters(), config.max_grad_norm)
    clip_grad_norm_(model.reduce_state.parameters(), config.max_grad_norm)

    optimizer.step()

    return loss.item()

In [0]:
def setup_train(model_file_path=None):
    r''' The model, optimizer, scheduler initialization'''
    model = Model(model_file_path)

    params = list(model.encoder.parameters()) + list(model.decoder.parameters()) + \
             list(model.reduce_state.parameters())
    initial_lr = config.lr_coverage if config.is_coverage else config.lr
    optimizer = Adagrad(params, lr=initial_lr, initial_accumulator_value=config.adagrad_init_acc)
    scheduler = StepLR(optimizer, step_size=7, gamma=0.5)

    start_iter, start_loss = 0, 0

    if model_file_path is not None:
        state = torch.load(model_file_path, map_location= lambda storage, location: storage)
        start_iter = state['iter']
        start_loss = state['current_loss']

        if not config.is_coverage:
            optimizer.load_state_dict(state['optimizer'])
            if use_cuda:
                for state in optimizer.state.values():
                    for k, v in state.items():
                        if torch.is_tensor(v):
                            state[k] = v.cuda()

    return model, optimizer, scheduler, start_iter, start_loss

# !!!!!!!!!!!!!!!!!!!!!!!!!!!Toy training, if you want to train NN!!!!!!!!!!!!!!!

In [0]:
# We trained the pointer generator model, so we change configuration 
config.pointer_gen = True
config.lr = 0.1
config.use_gpu = False

# Initialize model, optimizer, scheduler
model, optimizer, scheduler, start_iter, start_loss = setup_train()

# Initialize batcher "train"
batcher_train = Batcher(data_["train"], vocab, batch_size=32)

In [0]:
# Train for 1 iteration (toy_example=True)
model_point, train_history_point = train_model(model, batcher_train, optimizer, scheduler, 
                                               N_epoch=10, use_cuda=False, toy_example=True)

We can see that folders for the training sessions are created.

In [0]:
! ls weights

#### Load trained model

In [0]:
config.pointer_gen = True
model = Model(is_eval=True)
use_cuda = False

batcher = Batcher(data_["val"], vocab, batch_size=1)

In [0]:
checkpoint = torch.load("Abstractive-Summarization-pytorch/weights/train_2019-12-19_01:58:07/weights.loss_3.420.pt")
model.encoder.load_state_dict(checkpoint['encoder'])
model.decoder.load_state_dict(checkpoint['decoder'])
model.reduce_state.load_state_dict(checkpoint['reduce_state'])

#### Evaluate

In [0]:
def evaluate_batch(model, batch, n=1):
    enc_batch, enc_padding_mask, enc_lens, enc_batch_extend_vocab, extra_zeros, c_t_1, coverage = \
        get_input_from_batch(batch, use_cuda)
    dec_batch, dec_padding_mask, max_dec_len, dec_lens_var, target_batch = \
        get_output_from_batch(batch, use_cuda)

    n_predictions = []
    # n - number of predictions for one text
    for itr in range(n):
        # Encode batch and obtain hidden state for decoder part
        encoder_outputs, encoder_feature, encoder_hidden = model.encoder(enc_batch, enc_lens)
        s_t_1 = model.reduce_state(encoder_hidden)

        batch_prediction, batch_target = [], []
        
        # Init 0-th word as [PAD], whoch has index=0
        y_t_1 = torch.zeros(enc_batch.shape[0], dtype=int)
        
        # Iterate over the size abstract (100)
        for di in range(min(max_dec_len, config.max_dec_steps)):

            # Do not use target for prediction
            final_dist, s_t_1, c_t_1,attn_dist, p_gen, next_coverage = model.decoder(y_t_1, s_t_1,
                                                        encoder_outputs, encoder_feature, enc_padding_mask, c_t_1,
                                                        extra_zeros, enc_batch_extend_vocab, coverage, di)

            # Target 
            target = dec_batch[:, di]
            batch_target.append(target[0].item())

            # Prediction
            prediction = torch.argmax(final_dist, dim=1)
            batch_prediction.append(prediction[0].item())

            y_t_1 = prediction
            if y_t_1 >= 50004:
                y_t_1 *=0

            gold_probs = torch.gather(final_dist, 1, target.unsqueeze(1)).squeeze()
        
        # Prediction
        batch_prediction = " ".join([vocab.id2word(b) for b in batch_prediction if b < 50004])
        n_predictions.append(batch_prediction)

    # Target 
    batch_target = " ".join([vocab.id2word(b) for b in batch_target if b < 50004])

    # Text
    text = " ".join([vocab.id2word(b) for b in list(enc_batch[0].numpy()) if b < 50004])
        
    return text, batch_target, n_predictions


In [0]:
# Evaluate on 1000 validation samples

data_evaluation = []

for i in tqdm(range(1000)):
    
    batch = batcher.next_batch()
    text, batch_target, batch_prediction = evaluate_batch(model, batch, 1)
    
    sample = {"text": text, "abstract_target": batch_target, "batch_prediction": batch_prediction[0]}
    
    data_evaluation.append(sample)

In [0]:
with open("eval_discourse_for_TA", "w") as fp:
    json.dump(data_evaluation , fp)

#### Examples

In [0]:
for i in range(5):
    batch = batcher.next_batch()
    text, batch_target, batch_prediction = evaluate_batch(model, batch, 1)
    
    print("-"*50)
    print("\n --- TARGET --- ")
    print(batch_target)
    print("\n --- PREDICTION --- ")
    print(batch_prediction)

# Evaluate


Let's open our results

!!!!!! **IF YOU WANT TO TEST ON YOUR GENERATED JSONS, PLEASE REMOVE results/ folder FROM NEXT 2 CELLS** !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Thank you!

In [0]:
with open("results/pubmed_reviews_117M.json", "r") as read_file:
    pubmed_reviews_pretrain = json.load(read_file)

In [0]:
with open("results/pubmed_reviews_on_train_info.json", "r") as read_file:
    pubmed_reviews_on_train_info = json.load(read_file)

In [0]:
with open("results/eval_discourse.json", "r") as read_file:
    pubmed_reviews_discourse = json.load(read_file)

In [0]:
rouge = Rouge()

In [0]:
def calc_rouge(pubmed_reviews_pretrain):
    """
    calculate rouge metrics for every five generated summaries
    :pubmed_reviews_pretrain json file for evaluation
    """
    scores_all_rouge_1 = []
    scores_all_rouge_2 = []
    scores_all_rouge_l = []

    for num in range(1,1000+1):
        scores_one_rouge_1 =[]
        scores_one_rouge_2 =[]
        scores_one_rouge_l =[]
        for num_rev in range(5):
            hypothesis = pubmed_reviews_pretrain[f'review{num}']['generated'][num_rev]
            reference = pubmed_reviews_pretrain[f'review{num}']['summary']
            scores = rouge.get_scores(hypothesis, reference)[0]
            scores_one_rouge_1.append(scores['rouge-1']['f'])
            scores_one_rouge_2.append(scores['rouge-2']['f'])
            scores_one_rouge_l.append(scores['rouge-l']['f'])
        scores_all_rouge_1.append(scores_one_rouge_1)
        scores_all_rouge_2.append(scores_one_rouge_2)
        scores_all_rouge_l.append(scores_one_rouge_l)
    return scores_all_rouge_1, scores_all_rouge_2,scores_all_rouge_l

In [0]:
pretrain_rouge_1_all, pretrain_rouge_2_all,pretrain_rouge_l_all = calc_rouge(pubmed_reviews_pretrain)

# take the maximum score value from 5 summaries
pretrain_rouge_1 = np.mean([np.max(score) for score in pretrain_rouge_1_all])
pretrain_rouge_2 = np.mean([np.max(score) for score in pretrain_rouge_2_all])
pretrain_rouge_l = np.mean([np.max(score) for score in pretrain_rouge_l_all])

In [35]:
print('average score on pubmed_reviews_pretrain:')
print(f'rouge-1 = {pretrain_rouge_1}| rouge-2 = {pretrain_rouge_2}|rouge-l = {pretrain_rouge_l}')

average score on pubmed_reviews_pretrain:
rouge-1 = 0.13912234826900566| rouge-2 = 0.01804727956768799|rouge-l = 0.09981146943148786


In [0]:
retrain_rouge_1_all, retrain_rouge_2_all,retrain_rouge_l_all = calc_rouge(pubmed_reviews_on_train_info)
# take the maximum score value from 5 summaries
retrain_rouge_1 = np.mean([np.max(score) for score in retrain_rouge_1_all])
retrain_rouge_2 = np.mean([np.max(score) for score in retrain_rouge_2_all])
retrain_rouge_l = np.mean([np.max(score) for score in retrain_rouge_l_all])

In [55]:
print('average score on pubmed_reviews_finetune_train:')
print(f'rouge-1 = {retrain_rouge_1} | rouge-2 = {retrain_rouge_2} |rouge-l = {retrain_rouge_l}')

average score on pubmed_reviews_finetune_train:
rouge-1 = 0.15211777359614842 | rouge-2 = 0.021448975758018622 |rouge-l = 0.10986919676197734


Let's calculate metric on first summary for every review

In [0]:
hyps_discourse, refs_discourse = map(list, zip(*[[data['batch_prediction'], data['abstract_target']] for data in pubmed_reviews_discourse]))

In [0]:
scores_discourse = rouge.get_scores(hyps_discourse, refs_discourse, avg=True)

In [40]:
scores_discourse

{'rouge-1': {'f': 0.3450314093859501,
  'p': 0.3985205577595461,
  'r': 0.31686941259909085},
 'rouge-2': {'f': 0.13021916754204454,
  'p': 0.14686698589423725,
  'r': 0.12052901607778513},
 'rouge-l': {'f': 0.29400925667819094,
  'p': 0.35466393013384767,
  'r': 0.2824342449585647}}

In [0]:
hyps_pretrain, refs_pretrain = map(list, zip(*[[pubmed_reviews_pretrain[d]['generated'][0], pubmed_reviews_pretrain[d]['summary']] for d in pubmed_reviews_pretrain]))

In [0]:
hyps_on_train, refs_on_train = map(list, zip(*[[pubmed_reviews_on_train_info[d]['generated'][0], pubmed_reviews_on_train_info[d]['summary']] for d in pubmed_reviews_on_train_info]))

In [0]:
scores_pretrain = rouge.get_scores(hyps_pretrain, refs_pretrain, avg=True)

In [44]:
scores_pretrain

{'rouge-1': {'f': 0.10437971948822261,
  'p': 0.14661859050549553,
  'r': 0.1458612514629657},
 'rouge-2': {'f': 0.008606890631845252,
  'p': 0.013944246670931592,
  'r': 0.011568766070369373},
 'rouge-l': {'f': 0.07358603056641509,
  'p': 0.1358701220774641,
  'r': 0.13740603181686947}}

In [0]:
scores_on_train = rouge.get_scores(hyps_on_train, refs_on_train, avg=True)

In [46]:
scores_on_train

{'rouge-1': {'f': 0.11915119128935556,
  'p': 0.1650365255803232,
  'r': 0.16062293762822413},
 'rouge-2': {'f': 0.011356141595021641,
  'p': 0.018382549301293467,
  'r': 0.01408523122321655},
 'rouge-l': {'f': 0.08402063059451577,
  'p': 0.15433319472100834,
  'r': 0.15065229057669685}}

In [0]:
discourse_rouge_1_one = scores_discourse['rouge-1']['f']
discourse_rouge_2_one = scores_discourse['rouge-2']['f']
discourse_rouge_l_one = scores_discourse['rouge-l']['f']

In [0]:
pretrain_rouge_1_one = scores_pretrain['rouge-1']['f']
pretrain_rouge_2_one = scores_pretrain['rouge-2']['f']
pretrain_rouge_l_one = scores_pretrain['rouge-l']['f']

In [0]:
retrain_rouge_1_one = scores_on_train['rouge-1']['f']
retrain_rouge_2_one = scores_on_train['rouge-2']['f']
retrain_rouge_l_one = scores_on_train['rouge-l']['f']

Look at all of them

In [50]:
print('average score on pubmed_reviews_pretrain maximum from 5 summaries:')
print(f'rouge-1 = {pretrain_rouge_1}| rouge-2 = {pretrain_rouge_2}|rouge-l = {pretrain_rouge_l}')

average score on pubmed_reviews_pretrain maximum from 5 summaries:
rouge-1 = 0.13912234826900566| rouge-2 = 0.01804727956768799|rouge-l = 0.09981146943148786


In [56]:
print('average score on pubmed_reviews_finetune_train maximum from 5 summaries:')
print(f'rouge-1 = {retrain_rouge_1} | rouge-2 = {retrain_rouge_2} |rouge-l = {retrain_rouge_l}')

average score on pubmed_reviews_finetune_train maximum from 5 summaries:
rouge-1 = 0.15211777359614842 | rouge-2 = 0.021448975758018622 |rouge-l = 0.10986919676197734


In [52]:
print('average score on pubmed_reviews_pretrain use only 1 summary:')
print(f'rouge-1 = {pretrain_rouge_1_one}| rouge-2 = {pretrain_rouge_2_one}|rouge-l = {pretrain_rouge_l_one}')

average score on pubmed_reviews_pretrain use only 1 summary:
rouge-1 = 0.10437971948822261| rouge-2 = 0.008606890631845252|rouge-l = 0.07358603056641509


In [57]:
print('average score on pubmed_reviews_finetune_train use only 1 summary:')
print(f'rouge-1 = {retrain_rouge_1_one} | rouge-2 = {retrain_rouge_2_one} |rouge-l = {retrain_rouge_l_one}')

average score on pubmed_reviews_finetune_train use only 1 summary:
rouge-1 = 0.11915119128935556 | rouge-2 = 0.011356141595021641 |rouge-l = 0.08402063059451577


In [54]:
print('average score on pubmed_reviews_discourse use only 1 summary:')
print(f'rouge-1 = {discourse_rouge_1_one} | rouge-2 = {discourse_rouge_2_one} |rouge-l = {discourse_rouge_l_one}')

average score on pubmed_reviews_discourse use only 1 summary:
rouge-1 = 0.3450314093859501 | rouge-2 = 0.13021916754204454 |rouge-l = 0.29400925667819094


#Conclusion for GPT-2 models
In conclusion we would like to say that gpt-2 model which we trained in our data works better then just simple gpt-2 pretrained model.  As we generate 5 reviews for every review, we calculate metrics for each review and consider best score for each review set. we can see ROUGE metrics for fine tuned(futher trained model) in averege better in 0.03(3%). We can see that we have modest result because, originally GPT-2 fitted on large corpus(about 8 millions papers) of data in order to train. We have corpus from 100 000 papers, but training model can only on 40 000 papers, because we more papers cant storage in colab's RAM. Also  we have not much time fo trainig we trained only 5 000 epoches, training time more then 4 hours, it is not enough. For better results we think that we need fit model more then 100 000 epochs, and this is impossible in our case.

Results of discourse-aware attention model is better because we train our model only on pubmed dataset at whole(dont have a lot initial weights, then more free memory). That's why model fit dependences only for our dataset and predict it better. In the end we want to say, that for more fair comparison, we need fit gpt-2 model more epochs.